In [19]:
import pandas as pd
import numpy as np
import requests
#import sys
import os
from dotenv import load_dotenv
import matplotlib.pyplot as plt

In [36]:
class Movie:
    def __init__(self) -> None:        
        self._api_key =  os.getenv("TMDB_API_KEY") # Chave da API
        self._urlsystem = 'PASTA/'
        self.url = f"https://api.themoviedb.org/3/trending/movie/day?language=en-US"
        self.url_genre = "https://api.themoviedb.org/3/genre/movie/list?language=en"

    def extract(self):
        headers = {
             "accept": "application/json",
             "Authorization": f"Bearer {self._api_key}"
        }       
        movies_data = []

        for page in range(1,5):
            url = f"{self.url}?page={page}"
            response = requests.get(self.url, headers=headers)
        
            if response.status_code != 200:
                print(f"Erro na página {page}: {response.status_code}")
                break
        
            data = response.json()
        
            movies_data.extend(data['results']) 
        
        df = pd.DataFrame(movies_data)
        return df

    def extract_genre(self):
        headers = {
             "accept": "application/json",
             "Authorization": f"Bearer {self._api_key}"
        }         
        genre_data = []
        response = requests.get(self.url_genre, headers=headers)

        if response.status_code != 200:
            print(f"Erro na página {page}: {response.status_code}")
        
        genre_data = response.json()['genres']

        df = pd.DataFrame(genre_data)
        return df
        



In [37]:
movie = Movie()
df = movie.extract()
genre_df = movie.extract_genre()

In [38]:
len(df)

80

In [39]:
genre_df

,id,name
0,28,Action
1,12,Adventure
2,16,Animation
3,35,Comedy
4,80,Crime
5,99,Documentary
6,18,Drama
7,10751,Family
8,14,Fantasy
9,36,History
